In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import fiona
import pydash
import shapely.geometry
from shapely.geometry import Polygon, Point, shape
import pyproj
import numpy as np
from affine import Affine
from rasterio.features import rasterize

In [ ]:
from kk.dggs import DGGS
import kk.tools

dg = DGGS()

In [ ]:
poly, src_crs = kk.tools.load_polygons('/Users/kirill/wk/dggs/data/act_aea.shp')
poly = poly[0]
display(poly, src_crs)

In [ ]:
src_prj = pyproj.Proj(src_crs)
nx, ny = [np.asarray(a) for a in poly.boundary.xy]
lx, ly = pyproj.transform(src_prj, dg._prj_lonlat, nx, ny)

addr, w, h  = dg.compute_overlap(13, nx, ny, crs=src_crs)[0]
poly_m = Polygon(zip(*pyproj.transform(src_prj, dg._rhm, nx, ny)))

display( (addr, w, h) )
display(poly_m)

In [ ]:
tr, *_ = dg.pixel_coord_transform(addr, w, h, native=True, no_offset=True)

In [ ]:
mx, my = [np.asarray(a) for a in poly_m.boundary.xy]

poly_pix = Polygon(zip(*tr.inverse(mx, my)))
display(poly_pix)
[int(b) for b in poly_pix.bounds]

In [ ]:
im = rasterize([poly_pix], out_shape=(h,w))

plt.imshow(im)
plt.colorbar()

------------------------------------------

## keep

``` python
idx, x, y = dg.to_ixy(lx,ly, 13)
display(np.unique(idx))
poly_ = Polygon(zip(x,y))
display(poly_)


x1,y1, x2, y2 = poly_.bounds

w, h = int(x2-x1), int(y2-y1)
(w,h)

x1,y1

Affine.translation(x1,y1)

im = rasterize([poly_], out_shape=(h,w), transform=Affine.translation(x1,y1))
im.shape, im.sum()
```